In [ ]:
"""
Filename: main.ipynb
Author: Alexay Mehra
Date: 2025-09-12
Description: Main file for testing, contains the following:
- Image of the Morse Potential to fit
- Code block to run the optimization function
- Visuals to compare synthesized time evolution with true time evolution
"""

In [1]:
# Imports

import os, sys
sys.path.append(os.path.abspath("../src"))

import numpy as np
import scipy as sp
from qutip import coherent

import utils, optimization, visualization

**1. Define the Morse Potential Parameters:**
- *mass* = reduced mass of diatomic system
- *de* = well depth
- *b* = width parameter
- *x0* = equilibrium bond length
- *hbar* = reduced planck's constant (default 1 for easy units)

There are other values available based on these inputs:
- the *force constant* for the potential
- the *angular frequency* associated with the potential
- the *number of bounded states*
- the *maximum bounded quantum number*

In [ ]:
morse_potential = utils.MorsePotential(
    mass = 1,
    de = 10.0, 
    b = 1.0, 
    x0 = 0.5,
    hbar = 1
)

In [ ]:
# Plot the Morse Potential
morse_potential.plot_potential()

**2. Create the Backend for Optimization**


In [ ]:
# Select the size of the hilbert space and time to optimize over
backend = utils.Gates(
    dim = 40,
    time = 1
)

# Choose the gate depth to use
depth = 6

# Choose parameters for gates with configurable parameters not in the optimization
utils.AlwaysOnGate.config["omega"] = morse_potential.angular_freq
utils.AlwaysOnGate.config["chi"] = 0.01
utils.AlwaysOnGate.config["time"] = backend.time / depth

# Choose the gate sequence to use
sequence_template = [
    utils.ControlledDisplacement,
    utils.RotationGate,
    utils.ControlledCVRotation
    ]    

# generate the time evolution you want to match
morse_evolution, pos_evolution, _ = backend.morse_time_evolution(morse_potential)

**3. Choose the Initial State to Use:**

In [ ]:
# --- Choose Qubit Initial State ---
init_qubit_state = np.array([1, 0], dtype=complex)
qubit_state_index = 0

# --- Choose Qumode Initial State ---

# ----- Option 1: Fock state ---
#fock_state_index = 2
#init_qumode_state = np.zeros(N, dtype=complex)
#init_qumode_state[fock_state_index] = 1.0

# ----- Option 2: Coherent state ---
alpha = 1.0
init_qumode_state = coherent(backend.dim, alpha).full().flatten()

# Full Initial State is Set
init_state = np.kron(init_qubit_state, init_qumode_state)

**4. Run the Chosen Optimization Function**

In [ ]:
optimal_params = optimization.run_optimization(
    seq_template = sequence_template,
    backend = backend,
    d = depth,
    loss_function = optimization.fidelity_loss,
    target_evolution = morse_evolution,
    init_state = init_state,
    qubit_state_index = qubit_state_index,
    mp = morse_potential,
    mode = 'coordinate-descent',
    max_iterations = 5,
    info_type = 'position'
)

**4. Post Processing: Generate the Desired Visualizations**

In [ ]:
# Build the synthesized unitary and extract the hamiltonian
synthesized_evolution = optimization.build_sequence(
    params = optimal_params,
    seq_template = sequence_template,
    backend = backend,
    d = depth
)
synthesized_hamiltonian = (1j / backend.time) * sp.linalg.logm(synthesized_evolution)

In [ ]:
# Generate Wavefunction Graphics

target_time = 1 # End time to be visualized for the wavefunction
n_steps = 6     # number of time steps to visualize

# --- Generate Wavefunction Visualization ---
visualization.generate_wavefunc(
    init_state = init_state,
    target_time = target_time,
    n_steps = n_steps,
    gen_hamiltonian = synthesized_hamiltonian,
    backend = backend,
    morse_to_optimize = pos_evolution,
    mp = morse_potential
)

In [ ]:
# Generate Wigner Function Graphics

time_and_extra = 1     # end time to be visualized for the wigner function
steps = 6              # number of steps to visualize

# --- Generate Wigner Function Visualization ---
visualization.gen_wigfunc(
    init_state = init_state,
    time_and_extra = time_and_extra,
    steps = steps,
    generated_hamiltonian = synthesized_hamiltonian,
    backend = backend,
    morse_to_optimize = pos_evolution
)